In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2

In [6]:


# Constants
NEIGHBOR_OFFSETS = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # 4-connected neighbors


def region_growing(image, seed_points):
    # Initialize variables
    height, width = image.shape[:2]
    visited = np.zeros((height, width), dtype=bool)
    region = np.zeros((height, width), dtype=bool)
    queue = []

    # Add seed points to the queue
    for seed_point in seed_points:
        x, y = seed_point
        queue.append((x, y))
        region[x, y] = True

    # Region growing algorithm
    while len(queue) > 0:
        x, y = queue.pop(0)
        visited[x, y] = True

        # Check neighbors
        for offset in NEIGHBOR_OFFSETS:
            dx, dy = offset
            nx, ny = x + dx, y + dy

            # Check if neighbor is within image bounds
            if nx >= 0 and nx < height and ny >= 0 and ny < width:
                # Check if neighbor is unvisited and similar to the region
                if not visited[nx, ny] and similar(image[x, y], image[nx, ny]):
                    queue.append((nx, ny))
                    region[nx, ny] = True
                    visited[nx, ny] = True

    return region


def similar(pixel_a, pixel_b, threshold= 8):
    # Compare similarity based on pixel intensity difference
    return abs(int(pixel_a) - int(pixel_b)) <= threshold


# Load image
image_path = "R (1).jpeg" 
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Display image
cv2.imshow("Segmented Image", image)

# Capture seed points from mouse clicks
seed_points = []

def onclick(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        seed_points.append((y, x))

        # Perform region growing
        region = region_growing(image, seed_points)

        # Create segmented image
        segmented_image = np.zeros_like(image)
        segmented_image[region] = image[region]

        # Display segmented region
        cv2.imshow("Segmented Image", segmented_image)

cv2.setMouseCallback("Segmented Image", onclick)
cv2.waitKey(0)
cv2.destroyAllWindows()
